In [ ]:
!pip install vectorbt

In [1]:
import vectorbt as vbt
import yfinance as yf
import pandas as pd

# パラメータ
ns = 5   # 短期移動平均日数
nl = 25  # 長期移動平均日数

# データ取得（トヨタ）
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')

if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]  # 最初の銘柄の「Close」列だけ使う

# 移動平均の計算
smaS = vbt.MA.run(price, window=ns).ma
smaL = vbt.MA.run(price, window=nl).ma

# シグナル定義：クロスオーバー
entries = smaS > smaL
exits = smaS < smaL

# ポートフォリオ作成（シグナルによるバックテスト）
portfolio = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)

# 成績確認
print(portfolio.stats())
portfolio.plot()


/var/folders/x3/wccbb5_s6s183_4fl9f6ybdc0000gn/T/ipykernel_57892/1256612294.py:10: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')
/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packa

Start                         2021-01-03 15:00:00+00:00
End                           2022-03-29 15:00:00+00:00
Period                                              303
Start Value                                     10000.0
End Value                                  11227.374979
Total Return [%]                               12.27375
Benchmark Return [%]                          45.648892
Max Gross Exposure [%]                            100.0
Total Fees Paid                              441.516552
Max Drawdown [%]                               17.92058
Max Drawdown Duration                             191.0
Total Trades                                         11
Total Closed Trades                                  10
Total Open Trades                                     1
Open Trade PnL                               525.024916
Win Rate [%]                                       50.0
Best Trade [%]                                13.264976
Worst Trade [%]                               -6

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '88449ae8-c6e1-4f65-b504-4345789d1609',
              'x': array([datetime.datetime(2021, 1, 3, 15, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2021, 1, 4, 15, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2021, 1, 5, 15, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2022, 3, 27, 15, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 3, 28, 15, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 3, 29, 15, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1395.84973145, 1375.42614746, 1376.48254395, ..., 1992.92956543,
                

✅ vectorbtで移動平均線を最適化するサンプル

In [2]:
import vectorbt as vbt
import yfinance as yf
import pandas as pd

# データ取得
price = vbt.YFData.download('7203.T', start='2021-01-01', end='2022-03-31').get('Close')

# マルチインデックス対応
if isinstance(price, pd.DataFrame) and isinstance(price.columns, pd.MultiIndex):
    price = price.iloc[:, 0]

# パラメータの組み合わせ
ns_range = list(range(5, 30, 5))
nl_range = list(range(10, 55, 5))

# 条件：ns < nl を満たす組み合わせを作成
params = [(ns, nl) for ns in ns_range for nl in nl_range if ns < nl]

# エントリー/エグジットを生成する関数
def run_strategy(price, ns, nl):
    fast = vbt.MA.run(price, window=ns).ma
    slow = vbt.MA.run(price, window=nl).ma
    entries = fast > slow
    exits = fast < slow
    return entries, exits

# 結果格納用
portfolios = {}

# 各パラメータペアでポートフォリオ実行
for ns, nl in params:
    entries, exits = run_strategy(price, ns, nl)
    pf = vbt.Portfolio.from_signals(price, entries, exits, init_cash=10000, fees=0.002)
    portfolios[(ns, nl)] = pf.stats()['Total Return [%]']

# 結果を DataFrame に変換
result_df = pd.Series(portfolios).unstack()
result_df


/var/folders/x3/wccbb5_s6s183_4fl9f6ybdc0000gn/T/ipykernel_57892/3945072639.py:6: DeprecationWarning:

Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.

/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/cano/.pyenv/versions/3.13.1/lib/pytho

,10,15,20,25,30,35,40,45,50
5,7.795377,15.034989,28.790145,12.273750,-4.705989,9.196951,9.971648,20.414586,18.536514
10,NaN,14.625464,8.953078,13.235118,10.734765,5.532400,17.520727,27.771932,20.841595
15,NaN,NaN,-2.922098,4.085908,17.077655,26.509975,20.114824,20.263563,16.577497
20,NaN,NaN,NaN,14.137282,11.805757,14.240494,24.773816,21.457073,22.239735
25,NaN,NaN,NaN,NaN,8.780400,10.204418,10.130054,16.072046,19.534993


In [8]:
# ヒートマップで視覚化する
result_df.vbt.heatmap(title="Total Return [%] by ns/nl")

/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'type': 'heatmap',
              'uid': 'cb91df23-a439-4ed2-b9c8-094cd6021e5c',
              'x': array([10, 15, 20, 25, 30, 35, 40, 45, 50]),
              'y': array([ 5, 10, 15, 20, 25]),
              'z': array([[ 7.79538963, 15.03499481, 28.79015236, 12.27373404, -4.70602059,
                            9.19694146,  9.97166817, 20.41458737, 18.53648436],
                          [        nan, 14.62550076,  8.95302719, 13.23515671, 10.73473345,
    

In [3]:
# ヒートマップで表示（Jupyter対応、インタラクティブ）
result_df.vbt.heatmap(title="Total Return [%] by (ns, nl)", xaxis_title="Long MA (nl)", yaxis_title="Short MA (ns)")

/Users/cano/.pyenv/versions/3.13.1/lib/python3.13/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'type': 'heatmap',
              'uid': '1bf4977a-042c-41a4-a6cc-636f89f247ee',
              'x': array([10, 15, 20, 25, 30, 35, 40, 45, 50]),
              'y': array([ 5, 10, 15, 20, 25]),
              'z': array([[ 7.79537703, 15.03498858, 28.79014515, 12.27374979, -4.70598873,
                            9.19695099,  9.97164834, 20.41458622, 18.53651389],
                          [        nan, 14.62546426,  8.95307801, 13.23511839, 10.73476527,
    

In [4]:
# ベストペア

best_ns, best_nl = result_df.stack().idxmax()
best_return = result_df.stack().max()

print(f"🔥 最も良い組み合わせ: ns={best_ns}, nl={best_nl}（リターン: {best_return:.2f}%）")

🔥 最も良い組み合わせ: ns=5, nl=20（リターン: 28.79%）
